In [36]:
import pandas as pd
import numpy as np
from numpy import linalg as LA
import seaborn as sns
import matplotlib.pyplot as plt
import os
from utils import *


In [37]:
data = pd.read_parquet("data/vwap_bbo_data_with_mid_prices.parquet")
data.head(-5)

,time,Stock,bid_vwa,ask_vwa,vwap_mid_price
0,2008-04-23 14:26,PG.N,67.096526,67.110915,67.103720
1,2008-08-07 13:28,C.N,18.899127,18.911981,18.905554
2,2008-08-25 15:49,RTN.N,59.832404,59.834434,59.833419
3,2008-11-25 10:38,RTN.N,46.339335,46.404700,46.372018
4,2008-01-15 12:58,TWX.N,15.950000,15.960000,15.955000
...,...,...,...,...,...
1079303,2008-12-29 14:29,NKE.N,48.459904,48.508293,48.484098
1079304,2008-01-23 10:22,MO.N,72.784483,72.806719,72.795601
1079305,2008-09-30 10:26,RTN.N,53.249813,53.297344,53.273578
1079306,2008-09-19 13:47,EMC.N,12.842673,12.868889,12.855781


In [38]:
data['time'] = pd.to_datetime(data['time'])

data = data.sort_values(by=["time", "Stock"])

# Create pivot table
pivot_table = data.pivot(index="time", columns="Stock", values="vwap_mid_price")


pivot_table = pivot_table.sort_index(axis=1)  # Sorts the x-axis (time)

# Forward fill along the y-axis (axis=0 means filling down the rows)
pivot_table = pivot_table.fillna(method='bfill', axis=0)
pivot_table = pivot_table.fillna(method='ffill', axis=0)


print(pivot_table)


/var/folders/cg/lm106ggx1z59db7x0_wgfghm0000gn/T/ipykernel_9313/3659607899.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pivot_table = pivot_table.fillna(method='bfill', axis=0)
/var/folders/cg/lm106ggx1z59db7x0_wgfghm0000gn/T/ipykernel_9313/3659607899.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pivot_table = pivot_table.fillna(method='ffill', axis=0)


Stock                      C.N      EMC.N      HPQ.N      MDT.N       MO.N  \
time                                                                         
2008-01-02 09:30:00  29.690292  18.494578  50.502681  49.949452  75.641465   
2008-01-02 09:31:00  29.717395  18.476780  50.591862  49.956113  75.606653   
2008-01-02 09:32:00  29.807299  18.494106  50.602404  50.099181  75.819966   
2008-01-02 09:33:00  29.712021  18.565069  50.608779  50.110141  75.652896   
2008-01-02 09:34:00  29.609640  18.541359  50.606990  50.160005  75.574816   
...                        ...        ...        ...        ...        ...   
2008-12-31 15:56:00   6.896138  10.476895  36.428676  31.541954  15.105250   
2008-12-31 15:57:00   6.909711  10.476285  36.449454  31.559547  15.114467   
2008-12-31 15:58:00   6.915223  10.484686  36.439633  31.556522  15.124905   
2008-12-31 15:59:00   6.874229  10.464807  36.283635  31.520123  15.087841   
2008-12-31 16:00:00   6.855000  10.465000  36.283635  31.465000 

In [41]:
# Calculate returns
returns = pivot_table.pct_change() # simple linear returns
log_rets = np.log(1+returns)
log_rets

Stock,C.N,EMC.N,HPQ.N,MDT.N,MO.N,NKE.N,PG.N,RTN.N,SO.N,TWX.N,WFC.N
time,,,,,,,,,,,
2008-01-02 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-02 09:31:00,0.000912,-0.000963,0.001764,0.000133,-0.000460,0.000000,0.000000,0.000000,-0.003609,0.000811,-0.000938
2008-01-02 09:32:00,0.003021,0.000937,0.000208,0.002860,0.002817,-0.000876,-0.003091,0.001105,0.000964,0.001274,-0.000611
2008-01-02 09:33:00,-0.003202,0.003830,0.000126,0.000219,-0.002206,-0.000435,-0.002158,0.001351,0.001741,0.000458,-0.001478
2008-01-02 09:34:00,-0.003452,-0.001278,-0.000035,0.000995,-0.001033,-0.002415,-0.001420,-0.001554,0.000100,0.000088,0.000796
...,...,...,...,...,...,...,...,...,...,...,...
2008-12-31 15:56:00,0.001587,0.000825,0.001585,0.002357,0.000713,0.002661,0.000791,0.001690,0.001346,0.001972,0.002249
2008-12-31 15:57:00,0.001966,-0.000058,0.000570,0.000558,0.000610,-0.001188,-0.000034,0.000205,-0.000203,0.001971,0.001469
2008-12-31 15:58:00,0.000797,0.000802,-0.000269,-0.000096,0.000690,0.000213,0.000426,-0.000508,0.000811,0.001392,0.001469


In [42]:
# check if there are NaNs
if log_rets.isnull().values.any():
    print("There are NaN values in the pivot table.")
else:
    print("There are no NaN values in the pivot table.")

There are NaN values in the pivot table.


In [7]:
# fill NaNs with 0
log_rets = log_rets.fillna(0)
log_rets.head()

Stock,C.N,EMC.N,HPQ.N,MDT.N,MO.N,NKE.N,PG.N,RTN.N,SO.N,TWX.N,WFC.N
time,,,,,,,,,,,
2008-01-02 09:30:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-01-02 09:31:00,0.000912,-0.000963,0.001764,0.000133,-0.000460,0.000000,0.000000,0.000000,-0.003609,0.000811,-0.000938
2008-01-02 09:32:00,0.003021,0.000937,0.000208,0.002860,0.002817,-0.000876,-0.003091,0.001105,0.000964,0.001274,-0.000611
2008-01-02 09:33:00,-0.003202,0.003830,0.000126,0.000219,-0.002206,-0.000435,-0.002158,0.001351,0.001741,0.000458,-0.001478
2008-01-02 09:34:00,-0.003452,-0.001278,-0.000035,0.000995,-0.001033,-0.002415,-0.001420,-0.001554,0.000100,0.000088,0.000796


In [43]:
T_in = 60                       # lenght of the rolling window in minutes
start_period_in_sample = 8151   # 1st Febraury 2008
end_period_in_sample = 32417    # last day of April 2008
dT = 5
t0s = np.arange(start_period_in_sample, end_period_in_sample ,dT)    # 16031 is the index st the last date is 29/02/2008
t0s

array([ 8151,  8156,  8161, ..., 32406, 32411, 32416], shape=(4854,))

In [45]:
riks_in_sample = []
weights = []
means_in_sample = []

for t0 in t0s:
  
  t1 = t0 + T_in
  log_rets_cut = log_rets.iloc[t0:t1]
  
  # Compute means, covariance and correlation
  mean_values = log_rets_cut.mean()
  covariance_matrix = log_rets_cut.cov()
  
  # Compute the GMV weights
  weights_GVM = compute_weights_GVM(covariance_matrix)
  
  # compute risk in sample
  risk = np.dot(weights_GVM.T, np.dot(covariance_matrix, weights_GVM))
  
  # Append the results to the lists
  means_in_sample.append(mean_values)
  riks_in_sample.append(risk)
  weights.append(weights_GVM)
  
  

In [46]:
# Create DataFrame for means_in_sample
moving_avg_in_sample = pd.DataFrame(means_in_sample, index=log_rets.index[t0s+T_in], columns=pivot_table.columns)

# Create DataFrame for risks_in_sample
risks_df_in_sample = pd.DataFrame(riks_in_sample, index=log_rets.index[t0s+T_in], columns=['Risk'])

# Create DataFrame for weights
weights_df = pd.DataFrame(weights, index=log_rets.index[t0s+T_in], columns=pivot_table.columns)


In [47]:
weights_df.head()

Stock,C.N,EMC.N,HPQ.N,MDT.N,MO.N,NKE.N,PG.N,RTN.N,SO.N,TWX.N,WFC.N
time,,,,,,,,,,,
2008-02-01 09:30:00,-0.283125,0.094916,-0.228489,0.344422,0.333485,0.077326,0.234195,0.242347,0.092655,0.123783,-0.031515
2008-02-01 09:35:00,-0.285985,0.119753,-0.229875,0.427792,0.224376,0.023153,0.318659,0.283065,0.093964,0.043184,-0.018086
2008-02-01 09:40:00,-0.175625,0.068487,-0.174937,0.139305,0.341044,0.144504,0.283653,0.173302,0.144686,-0.000699,0.056280
2008-02-01 09:45:00,-0.166044,0.076587,-0.168531,0.091932,0.317723,0.129980,0.332181,0.167686,0.171520,-0.003405,0.050370
2008-02-01 09:50:00,-0.130619,0.052118,-0.166311,0.098280,0.336717,0.165337,0.251066,0.188424,0.166324,-0.015583,0.054248


In [48]:
risks_df_in_sample.head()

,Risk
time,
2008-02-01 09:30:00,2.427100e-07
2008-02-01 09:35:00,2.828104e-07
2008-02-01 09:40:00,3.619740e-07
2008-02-01 09:45:00,3.666299e-07
2008-02-01 09:50:00,3.678620e-07


### Out-of-Sample

In [49]:
T_in = 60  # lenght of the rolling window in minutes
start_period_out_sample = end_period_in_sample
end_period_out_sample = start_period_out_sample + (end_period_in_sample - start_period_in_sample) # keep same length as in-sample
t0s_out_sample = np.arange(start_period_out_sample, end_period_out_sample ,dT)    # 16031 is the index st the last date is 29/02/2008
t0s_out_sample

array([32417, 32422, 32427, ..., 56672, 56677, 56682], shape=(4854,))

In [50]:
riks_out_sample = []
moving_average_out_sample = []

for t1 in enumerate(t0s_out_sample):
  
  t2 = t1[1] + T_in
  log_rets_cut = log_rets.iloc[t1[1]:t2]
  
  # Compute means, covariance and correlation
  mean_values = log_rets_cut.mean()
  covariance_matrix = log_rets_cut.cov()
  
  # compute risk in sample
  risk = np.dot(weights[t1[0]].T, np.dot(covariance_matrix, weights[t1[0]]))
  
  # Append the results to the lists
  moving_average_out_sample.append(mean_values)
  riks_out_sample.append(risk)
  

In [51]:
# Create DataFrame for means_in_sample
moving_avg_out_sample = pd.DataFrame(moving_average_out_sample, index=log_rets.index[t0s_out_sample+T_in], columns=pivot_table.columns)

# Create DataFrame for risks_in_sample
risks_df_out_sample = pd.DataFrame(riks_out_sample, index=log_rets.index[t0s_out_sample+T_in], columns=['Risk'])


In [52]:
risks_df_out_sample.head()

,Risk
time,
2008-05-01 09:54:00,3.265855e-07
2008-05-01 09:59:00,3.553322e-07
2008-05-01 10:04:00,2.275061e-07
2008-05-01 10:09:00,2.123848e-07
2008-05-01 10:14:00,1.712682e-07


### Export into CSV

In [64]:
folder_name = "data/covariance_data"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created.")
else:
    print(f"Folder '{folder_name}' already exists.")


Folder 'data/covariance_data' created.


In [66]:
risks_df_in_sample.to_csv(f'{folder_name}/risks_in_sample.csv')
risks_df_out_sample.to_csv(f'{folder_name}/risks_out_sample.csv')
weights_df.to_csv(f'{folder_name}/weights_df.csv')
moving_avg_in_sample.to_csv(f'{folder_name}/moving_avg_in_sample.csv')
moving_avg_out_sample.to_csv(f'{folder_name}/moving_avg_out_sample.csv')